In [1]:
from math import *
import random

In [2]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

In [3]:
class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
#             raise ValueError, 'X coordinate out of bound'
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
#             raise ValueError, 'Y coordinate out of bound'
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
#             raise ValueError, 'Orientation must be in [0..2pi]'
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    def move(self, turn, forward):
        if forward < 0:
#             raise ValueError, 'Robot cant move backwards'
            raise ValueError('Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    def measurement_prob(self, measurement):
        # calculates how likely a measurement should be
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob

    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

In [4]:
def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [5]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.
myrobot = robot()
myrobot.set(30, 50, pi/2)
print(myrobot)
myrobot = myrobot.move(-pi/2, 15)
print(myrobot)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10)
print(myrobot)
print(myrobot.sense())

[x=30.0 y=50.0 orient=1.5707]
[x=45.0 y=50.0 orient=0.0]
[39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
[x=45.0 y=40.0 orient=4.7123]
[32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]


In [6]:
# Now add noise to your robot as follows:
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.
#
# Once again, your robot starts at 30, 50,
# heading north (pi/2), then turns clockwise
# by pi/2, moves 15 meters, senses,
# then turns clockwise by pi/2 again, moves
# 10 m, then senses again.
#
# Your program should print out the result of
# your two sense measurements.
#
forward_noise = 5.0
turn_noise = 0.1
sense_noise = 5.0

myrobot = robot()
myrobot.set_noise(forward_noise, turn_noise, sense_noise)
myrobot.set(30, 50, pi/2)
print(myrobot)
myrobot = myrobot.move(-pi/2, 15)
print(myrobot)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10)
print(myrobot)
print(myrobot.sense())

[x=30.0 y=50.0 orient=1.5707]
[x=44.691 y=51.508 orient=0.1023]
[44.08942010155568, 44.61597432272947, 37.870983693659205, 54.813346981710154]
[x=45.329 y=45.815 orient=4.8239]
[37.71293910149666, 51.137255149718584, 48.71957970255263, 46.388982865099614]


In [7]:
## initialize the particles
N = 1000
p = []
p = [robot() for i in range(N)]
print(len(p))
print(p[:10])

1000
[[x=66.219 y=39.766 orient=5.5582], [x=73.402 y=88.208 orient=2.5351], [x=23.424 y=63.161 orient=4.9126], [x=91.851 y=94.965 orient=6.0717], [x=1.7520 y=79.590 orient=1.6031], [x=14.797 y=74.940 orient=3.9640], [x=37.080 y=94.767 orient=5.0397], [x=48.613 y=77.045 orient=2.5678], [x=74.502 y=82.385 orient=1.8730], [x=21.968 y=28.116 orient=3.8272]]


In [8]:
# Now we want to simulate robot
# motion with our particles.
# Each particle should turn by 0.1
# and then move by 5. 
p = [pi.move(0.1, 5) for pi in p]
print(p[:10])

[[x=70.274 y=36.841 orient=5.6582], [x=69.030 y=90.633 orient=2.6351], [x=24.903 y=58.385 orient=5.0126], [x=96.820 y=94.409 orient=6.1717], [x=1.0923 y=84.546 orient=1.7031], [x=11.778 y=70.955 orient=4.0640], [x=39.152 y=90.217 orient=5.1397], [x=44.163 y=79.326 orient=2.6678], [x=72.545 y=86.986 orient=1.9730], [x=18.433 y=24.580 orient=3.9272]]


In [16]:
# Now we want to give weight to our 
# particles. This program will print a
# list of 1000 particle weights.
myrobot = robot()
myrobot.move(0.1, 5)
Z = myrobot.sense()   # this is the measurement, more precisely, the robot's measurements

N = 1000
particles = []
particles = [robot() for i in range(N)]
for p in particles:
    p.set_noise(0.05, 0.05, 5.0)

p2 = []
for i in range(N):
    p2.append(particles[i].move(0.1, 5.0))
particles = p2

p_weights = []
p_weights = [p.measurement_prob(Z) for p in particles]
# print(p_weights[:10])
p_weights[:10]

[2.503693439033037e-15,
 2.6397838049371558e-120,
 9.362062545755115e-45,
 4.776369421472468e-123,
 1.5935907792251127e-17,
 2.106312723124122e-110,
 7.53311886077369e-101,
 4.3803810380330645e-91,
 1.964347632090561e-55,
 5.216496641826595e-91]

In [18]:
# In this exercise, try to write a program that
# will resample particles according to their weights.
# Particles with higher weights should be sampled
# more frequently (in proportion to their weight).
# You should make sure that p3 contains a list with particles
# resampled according to their weights.
# Also, DO NOT MODIFY p.
sampled = [random.randint(0, 999) for i in range(N)]
p3 = [particles[s] for s in sampled]
p3[:10]

[[x=98.269 y=75.864 orient=1.8605],
 [x=31.116 y=42.695 orient=1.8537],
 [x=4.8557 y=3.3775 orient=2.9386],
 [x=94.393 y=11.028 orient=4.9682],
 [x=81.650 y=72.834 orient=3.3452],
 [x=66.876 y=43.367 orient=0.8848],
 [x=32.589 y=3.0317 orient=5.0407],
 [x=18.580 y=82.416 orient=3.8526],
 [x=41.994 y=84.850 orient=5.5788],
 [x=20.559 y=52.615 orient=4.1309]]

In [62]:
def sample_wheel(imp_weights, particles):
    # normalize the importance weights
    ###  don't need to normalize
#     weights_sum = sum(imp_weights)
#     normalized_weights = [w/weights_sum for w in imp_weights]
#     print(len(normalized_weights))
    
    ## the sample wheel
    picked_particle = []
#     index = int(random.uniform(0, N-1))
    index = int(random.random() * N)
#     print(index)
    beta = 0
#     w_max = max(normalized_weights)   # the max of normalized weights
    w_max = max(imp_weights)
    for i in range(N):
#         beta += random.uniform(0, 2*w_max)
        beta += random.random()* 2 * w_max
#         print(index)
#         if normalized_weights[index] < beta:
#             beta -= normalized_weights[index]
#             index += 1
#         else:
#             picked_particle.append(index)
        while imp_weights[index] < beta:
            beta -= imp_weights[index]
            index = (index + 1) % N
        picked_particle.append(particles[index])
    return picked_particle    

In [69]:
myrobot = robot()
N = 1000
particles = [robot() for i in range(N)]
for p in particles:
    p.set_noise(0.05, 0.05, 5.0)
print(eval(myrobot, particles))
print(" ======= filtering ======")
T = 10
for i in range(T):
    myrobot.move(0.1, 5)
    Z = myrobot.sense()   # this is the measurement, more precisely, the robot's measurements
    p_weights = [p.measurement_prob(Z) for p in particles]
    particles = sample_wheel(p_weights, particles)
    print(eval(myrobot, particles))
# particles

37.682846884123954
 ======= filtering ======
4.475604513735873
3.0082267784349996
2.5038821707501873
2.264698294083216
2.1073365329034575
2.0220679569551407
1.9123993936375094
1.862396005629863
1.7839882334086237
1.6877885989823902
